# Assignment #4

Solve systems of linear equations using the following numerical methods:

1. Gauss Elimination
2. LU Factorization
3. Jacobi Method
4. Gauss-Seidel Iterative Method
Each method must be implemented in MATLAB/ Python.

Compare the methods based on the following criteria:
- Accuracy: Compare the final numerical solutions.
- Computational Efficiency: Measure the number of operations/iterations required.
- Stability: Discuss if the method is sensitive to rounding errors.
- Applicability: When is each method preferable?

In [15]:
import pandas as pd
import sympy as sp
import numpy as np
from IPython.display import display, Math

from StopConditions.StopAtPlateau import StopAtPlateau
from utils.LaTeXTools import numpy_to_latex_gauss
from SolveEquations.LinearJacobiMethod import LinearJacobiMethod
from SolveEquations.LinearGaussSeidelMethod import LinearGaussSeidelMethod
from SolveEquations.LinearEquations import gauss_naive, lu_decomposition_linear_solver
from utils.LaTeXTools import df_to_latex
from pathlib import Path
import time

In [16]:
class timer:
    def __enter__(self):
        self.start = time.time()
        return self

    def __exit__(self, *args):
        self.end = time.time()
        # Convert seconds to milliseconds (1 second = 1000 milliseconds)
        self.interval = (self.end - self.start) * 1000

timing_data = dict()

In [17]:
# Linear equation in Matrix form
# columns are x_1, x_2,...,x_n, LHS
equations_augmented_matrix= sp.Matrix([
    [4,-1,2,11],
    [3,6,-1,8],
    [2,-1,5,7]
])
equations_coefficients_matrix= equations_augmented_matrix[:,:-1]
equations_lhs = equations_augmented_matrix[:,-1]
x_sym = sp.symbols(f'x_{{1:{equations_coefficients_matrix.shape[1]+1}}}')
x_sym_matrix = sp.Matrix(x_sym)
display(Math(
    f'{sp.latex(equations_coefficients_matrix)} '
    f'{sp.latex(x_sym_matrix)} = '
    f'{sp.latex(equations_lhs)}'
))

<IPython.core.display.Math object>

In [18]:
# Display Linear equations
sp.Eq(equations_coefficients_matrix * x_sym_matrix, equations_lhs)

Eq(Matrix([
[4*x_{1} - x_{2} + 2*x_{3}],
[3*x_{1} + 6*x_{2} - x_{3}],
[2*x_{1} - x_{2} + 5*x_{3}]]), Matrix([
[11],
[ 8],
[ 7]]))

## Gauss Elimination

In [19]:
gauss_elem_aug = equations_augmented_matrix.copy()
display(Math(numpy_to_latex_gauss(gauss_elem_aug)))

<IPython.core.display.Math object>

In [20]:
with timer() as t:
    gauss_naive_solution = gauss_naive(
        a=equations_coefficients_matrix.copy(),
        b=equations_lhs.copy(),
        verbose=True
    )

timing_data['Gauss'] = t.interval

**Forward Elimination**

<IPython.core.display.Math object>

<IPython.core.display.Math object>

**Back Substitution**

<IPython.core.display.Math object>

**Solution Vector**

<IPython.core.display.Math object>

In [21]:
gauss_naive_solution.evalf(10)

Matrix([
[ 2.582524272],
[0.1067961165],
[0.3883495146]])

## LU Decomposition

In [22]:
with timer() as t:
    lu_decomposition_solution = lu_decomposition_linear_solver(
        a=equations_coefficients_matrix.copy(),
        b=equations_lhs.copy(),
        verbose=True
    )

# Save the timing result
timing_data['LU Decomposition'] = t.interval

**LU Decomposition**

<IPython.core.display.Math object>

<IPython.core.display.Math object>

**$Ly=b$ forward substitution**

<IPython.core.display.Math object>

<IPython.core.display.Math object>

**$Lx=y$ back substitution**

<IPython.core.display.Math object>

**Solution Vector**

<IPython.core.display.Math object>

## LinearJacobiMethod

In [23]:
jacob_solver = LinearJacobiMethod(
    coefficients=np.array(equations_coefficients_matrix.evalf(15).copy(), dtype=float),
    lhs=np.array(equations_lhs.evalf(15), dtype=float).copy(),
    initial_guess=np.zeros((equations_coefficients_matrix.shape[0],1), dtype=float),
    stop_conditions=[
        StopAtPlateau(tracking='residual', patience=5, absolute_tolerance=1e-6),
    ]
)

with timer() as t:
    jacob_solver_df = jacob_solver.run()

timing_data['jacob method'] = t.interval

2025-03-24 19:57:40,103 - LinearJacobiMethod - INFO - Numerical.py:run:124 - Starting LinearJacobiMethod
2025-03-24 19:57:40,106 - LinearJacobiMethod - INFO - Numerical.py:initialize:70 - Initial state:{'x1': 0.0, 'x2': 0.0, 'x3': 0.0, 'residual': 15.297058540778355}
2025-03-24 19:57:40,107 - LinearJacobiMethod - INFO - Numerical.py:run:129 - Iteration 1 completed
Stop condition [StopAtPlateau: Stop when 'residual' plateaus (abs_tol=1e-06) for 5 iterations] NOT met: Not enough iterations to determine plateau
Stop condition [StopIfEqual: Stop when 'residual' equals 0 (abs_tol=1e-06) for 3 iterations] NOT met: Variable residual:15.2971 != 0 (abs diff: 15.2971 > 1e-06)
2025-03-24 19:57:40,109 - LinearJacobiMethod - INFO - Numerical.py:run:134 - State: 
{'x1': Decimal('2.75'), 'x2': Decimal('1.333333333333333333333333333'), 'x3': Decimal('1.4'), 'residual': 8.150749795093837}

2025-03-24 19:57:40,112 - LinearJacobiMethod - INFO - Numerical.py:run:129 - Iteration 2 completed
Stop condition 

In [24]:
df_to_latex(
    df=jacob_solver_df,
    filepath=Path(Path().cwd()/ 'Assignment4_latex' / 'jacob_solver.tex').resolve().__str__(),
    formatting=dict(
        x1=dict(header='$x_1$', format='0.6f'),
        x2=dict(header='$x_2$', format='0.6f'),
        x3=dict(header='$x_3$', format='0.6f'),
        residual=dict(header='$|r|$', format='0.6g'),
    ),
    precision=6,
    caption='Jacobi Method',
    label='tab:jacob_solver'
)

jacob_solver_df

2025-03-24 19:57:40,256 - utils.LaTeXTools - INFO - LaTeXTools.py:df_to_latex:90 - LaTeX table exported to /home/hhj/PycharmProjects/NumericalMethods/Examples/Assginment4/Assignment4_latex/jacob_solver.tex


,x1,x2,x3,residual
0,0.0,0.0,0.0,1.529706e+01
1,2.75,1.333333333333333333333333333,1.4,8.150750e+00
2,2.383333333333333333333333333,0.1916666666666666666666666667,0.5666666666666666666666666666,7.165698e-01
3,2.514583333333333333333333332,0.2361111111111111111111111113,0.4850000000000000000000000002,5.627973e-01
4,2.566527777777777777777777778,0.1568750000000000000000000007,0.4413888888888888888888888894,2.708812e-01
5,2.568524305555555555555555555,0.1236342592592592592592592592,0.404763888888888888888888889,6.930439e-02
6,2.57852662037037037037037037,0.116531828703703703703703704,0.3973171296296296296296296298,4.688575e-02
7,2.58047439236111111111111111,0.11028954475308641975308642,0.3918957175925925925925925928,1.583776e-02
8,2.581624527391975308641975308,0.1084120900848765432098765438,0.38986815200617283950617284,7.225221e-03
9,2.582168946518132716049382715,0.1074990949717078189300411527,0.3890326070601851851851851856,3.267572e-03


In [25]:
gauss_seidel_solver = LinearGaussSeidelMethod(
    coefficients=np.array(equations_coefficients_matrix.evalf(15).copy(), dtype=float),
    lhs=np.array(equations_lhs.evalf(15), dtype=float).copy(),
    initial_guess=np.zeros((equations_coefficients_matrix.shape[0],1), dtype=float),
    stop_conditions=[
        StopAtPlateau(tracking='residual', patience=5, absolute_tolerance=1e-6),
    ]
)

with timer() as t:
    gauss_seidel_solver_df = gauss_seidel_solver.run()

timing_data['gauss-seidel  method'] = t.interval

2025-03-24 19:57:40,380 - LinearGaussSeidelMethod - INFO - Numerical.py:run:124 - Starting LinearGaussSeidelMethod
2025-03-24 19:57:40,383 - LinearGaussSeidelMethod - INFO - Numerical.py:initialize:70 - Initial state:{'x1': 0.0, 'x2': 0.0, 'x3': 0.0, 'residual': 15.297058540778355}
2025-03-24 19:57:40,387 - LinearGaussSeidelMethod - INFO - Numerical.py:run:129 - Iteration 1 completed
Stop condition [StopAtPlateau: Stop when 'residual' plateaus (abs_tol=1e-06) for 5 iterations] NOT met: Not enough iterations to determine plateau
Stop condition [StopIfEqual: Stop when 'residual' equals 0 (abs_tol=1e-06) for 3 iterations] NOT met: Variable residual:15.2971 != 0 (abs diff: 15.2971 > 1e-06)
2025-03-24 19:57:40,389 - LinearGaussSeidelMethod - INFO - Numerical.py:run:134 - State: 
{'x1': 2.75, 'x2': -0.041666666666666664, 'x3': 0.2916666666666667, 'residual': 0.689706056551952}

2025-03-24 19:57:40,391 - LinearGaussSeidelMethod - INFO - Numerical.py:run:129 - Iteration 2 completed
Stop condit

In [26]:
df_to_latex(
    df=gauss_seidel_solver_df,
    filepath=Path(Path().cwd() / 'Assignment4_latex' / 'gauss_seidel_solver.tex').resolve().__str__(),
    formatting=dict(
        x1=dict(header='$x_1$', format='0.6f'),
        x2=dict(header='$x_2$', format='0.6f'),
        x3=dict(header='$x_3$', format='0.6f'),
        residual=dict(header='$|r|$', format='0.6g'),
    ),
    precision=6,
    caption='Gauss-Seidel Method',
    label='tab:gauss_seidel_solver'
)


gauss_seidel_solver_df

2025-03-24 19:57:40,561 - utils.LaTeXTools - INFO - LaTeXTools.py:df_to_latex:90 - LaTeX table exported to /home/hhj/PycharmProjects/NumericalMethods/Examples/Assginment4/Assignment4_latex/gauss_seidel_solver.tex


,x1,x2,x3,residual
0,0.000000,0.000000,0.000000,1.529706e+01
1,2.750000,-0.041667,0.291667,6.897061e-01
2,2.593750,0.085069,0.379514,1.005686e-01
3,2.581510,0.105830,0.388562,9.432324e-03
4,2.582177,0.107005,0.388530,1.238321e-03
5,2.582486,0.106845,0.388375,2.173700e-04
6,2.582524,0.106800,0.388350,2.437558e-05
7,2.582525,0.106796,0.388349,2.339823e-06
8,2.582524,0.106796,0.388349,4.260761e-07
9,2.582524,0.106796,0.388350,5.924110e-08


In [27]:
compare_df = pd.DataFrame(timing_data, index=['Time (ms)']).T
compare_df['error'] = [
    0,
    0,
    jacob_solver_df.loc[len(jacob_solver_df)-1, 'residual'],
    gauss_seidel_solver_df.loc[len(gauss_seidel_solver_df)-1,'residual']]
compare_df['iterations'] = [
    3+3,
    3+3,
    len(jacob_solver_df),
    len(gauss_seidel_solver_df)
]
compare_df

,Time (ms),error,iterations
Gauss,13.570070,0.000000e+00,6
LU Decomposition,21.283388,0.000000e+00,6
jacob method,97.787380,5.554834e-08,23
gauss-seidel method,54.306269,7.720835e-10,12


In [28]:
df_to_latex(
    df=compare_df,
    filepath=Path(Path().cwd() / 'Assignment4_latex' / 'compare_df.tex').resolve().__str__(),
    formatting=dict(
        Time=(dict(header='$Time (ms)$', format='0.6f')),
        error=(dict(header=r'$Error$', format='0.6g')),
        iterations=(dict(header='Iterations', format=str)),
    ),
    precision=6,
    caption='Comparison of Methods',
    label='tab:compare_df'
)

2025-03-24 19:57:40,889 - utils.LaTeXTools - INFO - LaTeXTools.py:df_to_latex:90 - LaTeX table exported to /home/hhj/PycharmProjects/NumericalMethods/Examples/Assginment4/Assignment4_latex/compare_df.tex


'\\begin{table}[h]\n\\centering\n\\begin{tabular}{lrll}\n\\toprule\n & Time (ms) & $Error$ & Iterations \\\\\n\\midrule\nGauss & 13.570070 & 0 & 6 \\\\\nLU Decomposition & 21.283388 & 0 & 6 \\\\\njacob method & 97.787380 & 5.55483e-08 & 23 \\\\\ngauss-seidel  method & 54.306269 & 7.72084e-10 & 12 \\\\\n\\bottomrule\n\\end{tabular}\n\n\\caption{Comparison of Methods}\n\\label{tab:compare_df}\n\\end{table}'